In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import os 
os.environ['OMP_NUM_THREADS'] = '4'
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
TRAIN_SAMPLE_SIZE = 800000
from datetime import time

C:\Users\hnpathak\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [160]:
morning_start = time(6)
morning_end = time(12)
afternoon_start = time(13)
afternoon_end = time(17)
evening_start = time(18)
evening_end = time(22)
night_start= time(23)
night_end= time(5)

In [161]:
final_test = pd.read_csv('./data/test/test.csv')
dff = final_test
dff.iloc[:,6] = pd.to_datetime(dff.iloc[:,6])
dff['time_of_day'] = dff.iloc[:,6].apply(lambda x: x.time())
#df['time_of_day'].head
dff['morning'] = (dff['time_of_day'] >= time(8,0)) & (dff['time_of_day'] <= time(12,0))
dff['afternoon'] = (dff['time_of_day'] >= time(12,1)) & (dff['time_of_day'] <= time(17,0))
dff['evening'] = (dff['time_of_day'] >= time(17,1)) & (dff['time_of_day'] <= time(20,0))
dff['night'] = (dff['time_of_day'] >= time(20,1)) & (dff['time_of_day'] <= time(23,59))
dff['late_night'] = (dff['time_of_day'] >= time(0,1)) & (dff['time_of_day'] <= time(8,0))
dff['night'] = dff['night'].astype(float)
dff['afternoon'] = dff['afternoon'].astype(float)
dff['late_night'] = dff['late_night'].astype(float)
dff['morning'] = dff['morning'].astype(float)
dff['evening'] = dff['evening'].astype(float)
test_df = dff
#del df
test_df.head()

,click_id,ip,app,device,os,channel,click_time,time_of_day,morning,afternoon,evening,night,late_night
0,0,5744,9,1,3,107,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
1,1,119901,9,1,3,466,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
2,2,72287,21,1,19,128,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
3,3,78477,15,1,13,111,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
4,4,123080,12,1,13,328,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0


In [163]:
del dff, test_df

In [162]:
test_df2 = test_df.iloc[:,[1,2,3,4,5,8,9,10,11,12]]
test_df2.head()

,ip,app,device,os,channel,morning,afternoon,evening,night,late_night
0,5744,9,1,3,107,0.0,0.0,0.0,0.0,1.0
1,119901,9,1,3,466,0.0,0.0,0.0,0.0,1.0
2,72287,21,1,19,128,0.0,0.0,0.0,0.0,1.0
3,78477,15,1,13,111,0.0,0.0,0.0,0.0,1.0
4,123080,12,1,13,328,0.0,0.0,0.0,0.0,1.0


In [170]:
Y_LABEL = 'y'                                   			        # Name of the variable to be predicted
#KEYS = [i for i in raw_data.keys().tolist() if i != Y_LABEL]	# Name of predictors
N_INSTANCES = df2.shape[0]                     			    # Number of instances
N_INPUT = x_train.shape[1]                      			    # Input size
N_CLASSES = 2                                    			    # Number of classes (output size)
TEST_SIZE = 0.1                                    			      # Test set size (% of dataset)
TRAIN_SIZE = int(N_INSTANCES * (1 - TEST_SIZE))     			    # Train size
LEARNING_RATE = 0.001                               			    # Learning rate
TRAINING_EPOCHS =  20                             			    # Number of epochs
BATCH_SIZE = 100                                    			    # Batch size
DISPLAY_STEP = 1                                    			    # Display progress each x epochs
HIDDEN_SIZE = 10          	                                   		      # Number of hidden neurons 256
ACTIVATION_FUNCTION_OUT = tf.nn.sigmoid                          # Last layer act fct
STDDEV = 0.1                                        			    # Standard deviation (for weights random init)
RANDOM_STATE = 100

n_input = N_INPUT                   # input n labels
n_hidden_1 = 20            # 1st layer
n_hidden_2 = 40            # 2nd layer
n_hidden_3 = 30            # 3rd layer
n_hidden_4 = 10           # 4th layer
n_classes = N_CLASSES               # output m classes

In [171]:
X = tf.placeholder(tf.float32, [None, n_input],name='X')
y = tf.placeholder(tf.float32, [None, n_classes],name='y')
dropout_keep_prob = tf.placeholder(tf.float32)


weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=STDDEV)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=STDDEV)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=STDDEV)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=STDDEV)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes],stddev=STDDEV)),                                   
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}    
   
def mlp(_X, _weights, _biases, dropout_keep_prob):
    layer1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), dropout_keep_prob)
    layer2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), dropout_keep_prob)
    layer3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), dropout_keep_prob)
    layer4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), dropout_keep_prob)
    out = ACTIVATION_FUNCTION_OUT(tf.add(tf.matmul(layer4, _weights['out']), _biases['out']))
    return out

In [172]:
pred = mlp(X, weights, biases, dropout_keep_prob)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) # softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

# Accuracy
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
confusion_matrix = tf.confusion_matrix(labels= tf.argmax(y, 1),predictions = tf.argmax(pred, 1), num_classes=2)
predictions =  tf.argmax(pred,1)

In [180]:
data_pos = pd.read_csv('data/positives/positives.csv',header=None)
print('Negative samples')
items = os.listdir('./data/negatives/')
count = 0
for item in items:
    count = count +1
    if count == 3:
        break
    #NEGATIVE data
    data_neg = pd.read_csv('data/negatives/'+str(item), header=None)
    print(data_neg.shape)
    #Combine one section of positive and negative
    data_combined =  pd.concat([data_pos, data_neg])
    data_combined = data_combined.sample(frac=1)
    print('data_combined: ', data_combined.shape)
    #keep interested columns
    data_f = data_combined.iloc[:,0:6]
    #data_f.astype(float)
    labels = data_combined.iloc[:,7:8]
    labels.astype(float)
    #n_values = np.max(labels) 
    labels_ = np.eye(2)[(labels.values).reshape(-1)]
    #convert the date and time
    df = data_f
    df.iloc[:,5] = pd.to_datetime(df.iloc[:,5])
    df['time_of_day'] = df.iloc[:,5].apply(lambda x: x.time())
    #df['time_of_day'].head
    df['morning'] = (df['time_of_day'] >= time(8,0)) & (df['time_of_day'] <= time(12,0))
    df['afternoon'] = (df['time_of_day'] >= time(12,1)) & (df['time_of_day'] <= time(17,0))
    df['evening'] = (df['time_of_day'] >= time(17,1)) & (df['time_of_day'] <= time(20,0))
    df['night'] = (df['time_of_day'] >= time(20,1)) & (df['time_of_day'] <= time(23,59))
    df['late_night'] = (df['time_of_day'] >= time(0,1)) & (df['time_of_day'] <= time(8,0))
    df['night'] = df['night'].astype(float)
    df['afternoon'] = df['afternoon'].astype(float)
    df['late_night'] = df['late_night'].astype(float)
    df['morning'] = df['morning'].astype(float)
    df['evening'] = df['evening'].astype(float)
    ###########################
    df2 = df.iloc[:,[0,1,2,3,4,7,8,9,10,11]]
    df2.head()
    df2.astype(float).head()
    #split the data set
    x_train, x_test, y_train, y_test = train_test_split(df2,labels_,test_size = 0.01,random_state = 100)
    x_train.astype(float)
    x_test.astype(float)
    y_train.astype(float)
    y_test.astype(float)
    ##################################
    ##################################
    ###################################
    tf.set_random_seed(1234)
    #saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for epoch in range(TRAINING_EPOCHS):
            avg_cost = 0.
            total_batch = int(x_train.shape[0] / BATCH_SIZE)
            X_batches = np.array_split(x_train, total_batch)
            Y_batches = np.array_split(y_train, total_batch)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys =  X_batches[i], Y_batches[i]
                # Fit using batched data
                sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.9})
                # Calculate average cost
                avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
            # Display progress
            if epoch % DISPLAY_STEP == 0:
                print("Epoch: %03d/%03d cost: %.9f" % (epoch, TRAINING_EPOCHS, avg_cost))
                train_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
                print("Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accuracy, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
        confusion = sess.run(confusion_matrix, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
        print("Test accuracy: %.3f" % (test_acc))
        answer  = sess.run(predictions, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
        final_test_acc = sess.run(predictions, feed_dict={X: test_df2.values, y: y_test, dropout_keep_prob:1.})
        #print(answer)
        np.savetxt('./m1/final_test_'+item+'.csv',final_test_acc)
        #np.savetxt('./m1/answer', answer)
        print(confusion)
        #saver.save(sess, './m1/model/')


Negative samples
(737196, 8)
data_combined:  (1194042, 8)
Epoch: 000/005 cost: 0.695697863
Training accuracy: 0.610
Epoch: 001/005 cost: 0.695888526
Training accuracy: 0.610
Epoch: 002/005 cost: 0.695866950
Training accuracy: 0.610


KeyboardInterrupt: 

In [2]:
# data is divided into positive negative samples and 7 gb file of negatives is further divided into 250 splits. 
# $ split -n l/250 negatives.csv
print('Negative samples')
data_neg = pd.read_csv('data/negatives/xaa', header=None)
print(data_neg.head())
print(data_neg.shape)
print('\n Positive samples')
data_pos = pd.read_csv('data/positives/positives.csv',header=None)
print(data_pos.head())
print(data_pos.shape)

Negative samples
        0   1  2   3    4                    5   6  7
0   83230   3  1  13  379  2017-11-06 14:32:21 NaN  0
1   17357   3  1  19  379  2017-11-06 14:33:34 NaN  0
2   35810   3  1  13  379  2017-11-06 14:34:12 NaN  0
3   45745  14  1  13  478  2017-11-06 14:34:52 NaN  0
4  161007   3  1  13  379  2017-11-06 14:35:08 NaN  0
(737196, 8)

 Positive samples
        0   1  2   3    4                    5                    6  7
0  204158  35  1  13   21  2017-11-06 15:41:07  2017-11-07 08:17:19  1
1   29692   9  1  22  215  2017-11-06 16:00:02  2017-11-07 10:05:22  1
2   64516  35  1  13   21  2017-11-06 16:00:02  2017-11-06 23:40:50  1
3  172429  35  1  46  274  2017-11-06 16:00:03  2017-11-07 00:55:29  1
4  199085  35  1  13  274  2017-11-06 16:00:04  2017-11-06 23:04:54  1
(456846, 8)


In [3]:
# Build x_train, y_tarin, x_test, y_test
data_combined =  pd.concat([data_pos, data_neg])
data_combined = data_combined.sample(frac=1)
print('data_combined: ', data_combined.shape)


data_combined:  (1194042, 8)


In [41]:
data_f = data_combined.iloc[:,0:6]

#data_f.astype(float)
labels = data_combined.iloc[:,7:8]
labels.astype(float)
type(labels)
#n_values = np.max(labels) 
labels_ = np.eye(2)[(labels.values).reshape(-1)]
labels_.shape
# labels = labels.values
# enc = OneHotEncoder()
# enc.fit(labels)
# labels= enc.transform(labels)
# labels.shape
#del data_combined, data_neg, data_pos
# y = np.zeros((data_combined.shape[0], 2))
# y[np.arange(data_combined.shape[0]), y] = 1

(1194042, 2)

In [61]:
from datetime import time
df = data_f
morning_start = time(6)
morning_end = time(12)
afternoon_start = time(13)
afternoon_end = time(17)
evening_start = time(18)
evening_end = time(22)
night_start= time(23)
night_end= time(5)
df.head()
df.iloc[:,5] = pd.to_datetime(df.iloc[:,5])
df['time_of_day'] = df.iloc[:,5].apply(lambda x: x.time())
#df['time_of_day'].head

df['morning'] = (df['time_of_day'] >= time(8,0)) & (df['time_of_day'] <= time(12,0))
df['afternoon'] = (df['time_of_day'] >= time(12,1)) & (df['time_of_day'] <= time(17,0))
df['evening'] = (df['time_of_day'] >= time(17,1)) & (df['time_of_day'] <= time(20,0))
df['night'] = (df['time_of_day'] >= time(20,1)) & (df['time_of_day'] <= time(23,59))
df['late_night'] = (df['time_of_day'] >= time(0,1)) & (df['time_of_day'] <= time(8,0))
df['night'] = df['night'].astype(float)
df['afternoon'] = df['afternoon'].astype(float)
df['late_night'] = df['late_night'].astype(float)
df['morning'] = df['morning'].astype(float)
df['evening'] = df['evening'].astype(float)
df.head()


,0,1,2,3,4,5,time_of_day,morning,afternoon,evening,night,late_night
392605,42424,9,1,17,466,2017-11-09 07:58:17,07:58:17,0.0,0.0,0.0,0.0,1.0
276471,45745,20,1,13,259,2017-11-08 12:08:02,12:08:02,0.0,1.0,0.0,0.0,0.0
258022,46542,14,1,13,489,2017-11-06 16:05:05,16:05:05,0.0,1.0,0.0,0.0,0.0
672377,131715,18,1,17,121,2017-11-06 16:14:00,16:14:00,0.0,1.0,0.0,0.0,0.0
327700,80975,14,1,53,463,2017-11-06 16:06:16,16:06:16,0.0,1.0,0.0,0.0,0.0


In [64]:
df2 = df.iloc[:,[0,1,2,3,4,7,8,9,10,11]]
df2.head()
df2.astype(float).head()
del df

,0,1,2,3,4,morning,afternoon,evening,night,late_night
392605,42424.0,9.0,1.0,17.0,466.0,0.0,0.0,0.0,0.0,1.0
276471,45745.0,20.0,1.0,13.0,259.0,0.0,1.0,0.0,0.0,0.0
258022,46542.0,14.0,1.0,13.0,489.0,0.0,1.0,0.0,0.0,0.0
672377,131715.0,18.0,1.0,17.0,121.0,0.0,1.0,0.0,0.0,0.0
327700,80975.0,14.0,1.0,53.0,463.0,0.0,1.0,0.0,0.0,0.0


In [65]:
x_train, x_test, y_train, y_test = train_test_split(df2,labels_,test_size = 0.1,random_state = 100)
x_train.astype(float)
x_test.astype(float)
y_train.astype(float)
y_test.astype(float)

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [80]:
final_test = pd.read_csv('./data/test/test.csv')
final_test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [94]:
#del df
from datetime import time
df = final_test
morning_start = time(6)
morning_end = time(12)
afternoon_start = time(13)
afternoon_end = time(17)
evening_start = time(18)
evening_end = time(22)
night_start= time(23)
night_end= time(5)

df.iloc[:,6] = pd.to_datetime(df.iloc[:,6])
df['time_of_day'] = df.iloc[:,6].apply(lambda x: x.time())
#df['time_of_day'].head
df['morning'] = (df['time_of_day'] >= time(8,0)) & (df['time_of_day'] <= time(12,0))
df['afternoon'] = (df['time_of_day'] >= time(12,1)) & (df['time_of_day'] <= time(17,0))
df['evening'] = (df['time_of_day'] >= time(17,1)) & (df['time_of_day'] <= time(20,0))
df['night'] = (df['time_of_day'] >= time(20,1)) & (df['time_of_day'] <= time(23,59))
df['late_night'] = (df['time_of_day'] >= time(0,1)) & (df['time_of_day'] <= time(8,0))
df['night'] = df['night'].astype(float)
df['afternoon'] = df['afternoon'].astype(float)
df['late_night'] = df['late_night'].astype(float)
df['morning'] = df['morning'].astype(float)
df['evening'] = df['evening'].astype(float)
print(df.head())
test_df = df
#del df
test_df.head()

   click_id      ip  app  device  os  channel          click_time time_of_day  \
0         0    5744    9       1   3      107 2017-11-10 04:00:00    04:00:00   
1         1  119901    9       1   3      466 2017-11-10 04:00:00    04:00:00   
2         2   72287   21       1  19      128 2017-11-10 04:00:00    04:00:00   
3         3   78477   15       1  13      111 2017-11-10 04:00:00    04:00:00   
4         4  123080   12       1  13      328 2017-11-10 04:00:00    04:00:00   

   morning  afternoon  evening  night  late_night  
0      0.0        0.0      0.0    0.0         1.0  
1      0.0        0.0      0.0    0.0         1.0  
2      0.0        0.0      0.0    0.0         1.0  
3      0.0        0.0      0.0    0.0         1.0  
4      0.0        0.0      0.0    0.0         1.0  


,click_id,ip,app,device,os,channel,click_time,time_of_day,morning,afternoon,evening,night,late_night
0,0,5744,9,1,3,107,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
1,1,119901,9,1,3,466,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
2,2,72287,21,1,19,128,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
3,3,78477,15,1,13,111,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0
4,4,123080,12,1,13,328,2017-11-10 04:00:00,04:00:00,0.0,0.0,0.0,0.0,1.0


In [100]:
test_df2 = test_df.iloc[:,[1,2,3,4,5,8,9,10,11,12]]
test_df2.head()

,ip,app,device,os,channel,morning,afternoon,evening,night,late_night
0,5744,9,1,3,107,0.0,0.0,0.0,0.0,1.0
1,119901,9,1,3,466,0.0,0.0,0.0,0.0,1.0
2,72287,21,1,19,128,0.0,0.0,0.0,0.0,1.0
3,78477,15,1,13,111,0.0,0.0,0.0,0.0,1.0
4,123080,12,1,13,328,0.0,0.0,0.0,0.0,1.0


In [179]:
test_df2.iloc[:,6:10].sum(axis=1)

0           1.0
1           1.0
2           1.0
3           1.0
4           1.0
5           1.0
6           1.0
7           1.0
8           1.0
9           1.0
10          1.0
11          1.0
12          1.0
13          1.0
14          1.0
15          1.0
16          1.0
17          1.0
18          1.0
19          1.0
20          1.0
21          1.0
22          1.0
23          1.0
24          1.0
25          1.0
26          1.0
27          1.0
28          1.0
29          1.0
           ... 
18790439    1.0
18790440    1.0
18790441    1.0
18790442    1.0
18790443    1.0
18790444    1.0
18790445    1.0
18790446    1.0
18790447    1.0
18790448    1.0
18790449    1.0
18790450    1.0
18790451    1.0
18790452    1.0
18790453    1.0
18790454    1.0
18790455    1.0
18790456    1.0
18790457    1.0
18790458    1.0
18790459    1.0
18790460    1.0
18790461    1.0
18790462    1.0
18790463    1.0
18790464    1.0
18790465    1.0
18790466    1.0
18790467    1.0
18790468    1.0
Length: 18790469, dtype:

In [95]:
del df

In [66]:
Y_LABEL = 'y'                                   			        # Name of the variable to be predicted
#KEYS = [i for i in raw_data.keys().tolist() if i != Y_LABEL]	# Name of predictors
N_INSTANCES = df2.shape[0]                     			    # Number of instances
N_INPUT = x_train.shape[1]                      			    # Input size
N_CLASSES = 2                                    			    # Number of classes (output size)
TEST_SIZE = 0.1                                    			      # Test set size (% of dataset)
TRAIN_SIZE = int(N_INSTANCES * (1 - TEST_SIZE))     			    # Train size
LEARNING_RATE = 0.001                               			    # Learning rate
TRAINING_EPOCHS =   1                             			    # Number of epochs
BATCH_SIZE = 100                                    			    # Batch size
DISPLAY_STEP = 1                                    			    # Display progress each x epochs
HIDDEN_SIZE = 10          	                                   		      # Number of hidden neurons 256
ACTIVATION_FUNCTION_OUT = tf.nn.sigmoid                          # Last layer act fct
STDDEV = 0.1                                        			    # Standard deviation (for weights random init)
RANDOM_STATE = 100   		

In [67]:
n_input = N_INPUT                   # input n labels
n_hidden_1 = 20            # 1st layer
n_hidden_2 = 40            # 2nd layer
n_hidden_3 = 30            # 3rd layer
n_hidden_4 = 10           # 4th layer
n_classes = N_CLASSES               # output m classes

In [68]:

X = tf.placeholder(tf.float32, [None, n_input],name='X')
y = tf.placeholder(tf.float32, [None, n_classes],name='y')
dropout_keep_prob = tf.placeholder(tf.float32)


weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=STDDEV)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=STDDEV)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3],stddev=STDDEV)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4],stddev=STDDEV)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes],stddev=STDDEV)),                                   
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}    
   
def mlp(_X, _weights, _biases, dropout_keep_prob):
    layer1 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])), dropout_keep_prob)
    layer2 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer1, _weights['h2']), _biases['b2'])), dropout_keep_prob)
    layer3 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer2, _weights['h3']), _biases['b3'])), dropout_keep_prob)
    layer4 = tf.nn.dropout(tf.nn.relu(tf.add(tf.matmul(layer3, _weights['h4']), _biases['b4'])), dropout_keep_prob)
    out = ACTIVATION_FUNCTION_OUT(tf.add(tf.matmul(layer4, _weights['out']), _biases['out']))
    return out



In [69]:
pred = mlp(X, weights, biases, dropout_keep_prob)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) # softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost)

# Accuracy
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
confusion_matrix = tf.confusion_matrix(labels= tf.argmax(y, 1),predictions = tf.argmax(pred, 1), num_classes=2)
predictions =  tf.argmax(pred,1)

In [101]:
tf.set_random_seed(1234)
saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for epoch in range(TRAINING_EPOCHS):
        avg_cost = 0.
        total_batch = int(x_train.shape[0] / BATCH_SIZE)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop over all batches
        for i in range(total_batch):
#             randidx = np.random.randint(int(TRAIN_SIZE), size = BATCH_SIZE)
#             batch_xs = x_tr[randidx, :]
#             batch_ys = labels[randidx, :]
            batch_xs, batch_ys =  X_batches[i], Y_batches[i]
            # Fit using batched data
            sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob: 0.9})
            # Calculate average cost
            avg_cost += sess.run(cost, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})/total_batch
        # Display progress
        if epoch % DISPLAY_STEP == 0:
            print("Epoch: %03d/%03d cost: %.9f" % (epoch, TRAINING_EPOCHS, avg_cost))
            train_acc = sess.run(accuracy, feed_dict={X: batch_xs, y: batch_ys, dropout_keep_prob:1.})
            print("Training accuracy: %.3f" % (train_acc))
    test_acc = sess.run(accuracy, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    confusion = sess.run(confusion_matrix, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    print("Test accuracy: %.3f" % (test_acc))
    answer  = sess.run(predictions, feed_dict={X: x_test, y: y_test, dropout_keep_prob:1.})
    final_test_acc = sess.run(predictions, feed_dict={X: test_df2.values, y: y_test, dropout_keep_prob:1.})
    #print(answer)
    np.savetxt('./m1/final_test_acc', final_test_acc)
    np.savetxt('./m1/answer', answer)
    print(confusion)
    saver.save(sess, './m1/model/')


Epoch: 000/001 cost: 0.583111460
Training accuracy: 0.810
Test accuracy: 0.787
[[66803  6909]
 [18532 27161]]


In [103]:
temp = np.loadtxt('./m1/final_test_acc')
np.savetxt('./m1/final_test_acc.csv', temp, delimiter=",")

In [ ]:
temp.head()

In [151]:
temp = temp.astype(np.int32)
final_test['click_id'] = final_test['click_id'].astype(np.int32)
a = np.vstack((final_test['click_id'].values,temp))
a = a.T
a = np.int32(a)
#a[:5]
np.savetxt('./m1/final_test_acc1.csv', a, fmt='%i', delimiter=",")